In [26]:
#pip install imutils

In [1]:
from imutils.video import VideoStream
from imutils.video import FPS
import numpy as np
import time
import imutils
import cv2
import os

In [2]:
protoPath = './WebCamProto/deploy.prototxt'
modelPath = './WebCamProto/res10_300x300_ssd_iter_140000.caffemodel'
detector = cv2.dnn.readNetFromCaffe(protoPath, modelPath)
detector

<dnn_Net 00000230EF46A5F0>

In [3]:
vs = VideoStream(src = 0 ).start()
time.sleep(2.0)
fps = FPS().start()

In [4]:
while True:
    frame = vs.read()
    frame = imutils.resize(frame, width = 600)
    (h,w) = frame.shape[:2]
    
    imgBlob = cv2.dnn.blobFromImage(
        cv2.resize( frame, (300,300)),
        1.0, 
        (300,300),
        (104.0,177.0,123.0), 
        swapRB=False,
    crop=False)
    detector.setInput(imgBlob)
    detections = detector.forward()
    
    for i in range(0,detections.shape[2]):
        confidence = detections[0,0,i,2]
        if confidence > 0.5 :
            box = detections[0,0,i,3:7] *np.array([w,h,w,h])
            (sx,sy, ex,ey) = box.astype('int')
            
            face = frame[sy:ey, sx:ex]
            (fh,fw) = face.shape[:2]
            if fw < 20 or fh < 20 :
                continue
            
            faceBlob = cv2.dnn.blobFromImage(face, 
                                             1.0/255,
                                             (96,96),
                                             (0,0,0), 
                                             swapRB=True,
                                             crop=False )
            y = sy -10 if sy -10 > 10 else sy +10
#             face_image = cv2.GaussianBlur(face,(99,99),30)
#             frame[sy:ey,sx:ex] = face_image
            
            cv2.rectangle(frame, (sx,sy),(ex,ey),(0,0,255),2)
    fps.update()
    cv2.imshow('Frame',frame)
    key = cv2.waitKey(1) & 0xff
    if key == ord('q') :
        break;
fps.stop()
cv2.destroyAllWindows()
vs.stop()